# AutoGluon Tabular - In Depth

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/tabular-indepth.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/tabular-indepth.ipynb)


**Tip**: If you are new to AutoGluon, review [Predicting Columns in a Table - Quick Start](tabular-quick-start.ipynb) to learn the basics of the AutoGluon API. To learn how to add your own custom models to the set that AutoGluon trains, tunes, and ensembles, review [Adding a custom model to AutoGluon](advanced/tabular-custom-model.ipynb).

This tutorial describes how you can exert greater control when using AutoGluon's `fit()` or `predict()`. Recall that to maximize predictive performance, you should first try `TabularPredictor()` and `fit()` with all default arguments.  Then, consider non-default arguments for `TabularPredictor(eval_metric=...)`, and `fit(presets=...)`.  Later, you can experiment with other arguments to fit() covered in this in-depth tutorial like `hyperparameter_tune_kwargs`, `hyperparameters`, `num_stack_levels`, `num_bag_folds`, `num_bag_sets`, etc.

Using the same census data table as in the [Predicting Columns in a Table - Quick Start](tabular-quick-start.ipynb) tutorial, we'll now predict the `occupation` of an individual - a multiclass classification problem. Start by importing AutoGluon's TabularPredictor and TabularDataset, and loading the data.

In [1]:
!pip install autogluon.tabular[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.6 M

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

import numpy as np

train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 1000  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
print(train_data.head())

label = 'occupation'
print("Summary of occupation column: \n", train_data['occupation'].describe())

test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])  # delete label column

metric = 'accuracy' # we specify eval-metric just for demo

       age workclass  fnlwgt      education  education-num  \
6118    51   Private   39264   Some-college             10   
23204   58   Private   51662           10th              6   
29590   40   Private  326310   Some-college             10   
18116   37   Private  222450        HS-grad              9   
33964   62   Private  109190      Bachelors             13   

            marital-status        occupation    relationship    race      sex  \
6118    Married-civ-spouse   Exec-managerial            Wife   White   Female   
23204   Married-civ-spouse     Other-service            Wife   White   Female   
29590   Married-civ-spouse      Craft-repair         Husband   White     Male   
18116        Never-married             Sales   Not-in-family   White     Male   
33964   Married-civ-spouse   Exec-managerial         Husband   White     Male   

       capital-gain  capital-loss  hours-per-week  native-country   class  
6118              0             0              40   United-State

## Specifying hyperparameters and tuning them

**Note: We don't recommend doing hyperparameter-tuning with AutoGluon in most cases**. AutoGluon achieves its best performance without hyperparameter tuning and simply specifying `presets="best_quality"`.

We first demonstrate hyperparameter-tuning and how you can provide your own validation dataset that AutoGluon internally relies on to: tune hyperparameters, early-stop iterative training, and construct model ensembles. One reason you may specify validation data is when future test data will stem from a different distribution than training data (and your specified validation data is more representative of the future data that will likely be encountered).

 If you don't have a strong reason to provide your own validation dataset, we recommend you omit the `tuning_data` argument. This lets AutoGluon automatically select validation data from your provided training set (it uses smart strategies such as stratified sampling).  For greater control, you can specify the `holdout_frac` argument to tell AutoGluon what fraction of the provided training data to hold out for validation.

**Caution:** Since AutoGluon tunes internal knobs based on this validation data, performance estimates reported on this data may be over-optimistic. For unbiased performance estimates, you should always call `predict()` on a separate dataset (that was never passed to `fit()`), as we did in the previous **Quick-Start** tutorial. We also emphasize that most options specified in this tutorial are chosen to minimize runtime for the purposes of demonstration and you should select more reasonable values in order to obtain high-quality models.

`fit()` trains neural networks and various types of tree ensembles by default. You can specify various hyperparameter values for each type of model. For each hyperparameter, you can either specify a single fixed value, or a search space of values to consider during hyperparameter optimization. Hyperparameters which you do not specify are left at default settings chosen automatically by AutoGluon, which may be fixed values or search spaces.

Refer to the [Search Space documentation](../../api/autogluon.common.space.rst) to learn more about AutoGluon search space.

In [3]:
from autogluon.common import space

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 2*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}  # Refer to TabularPredictor.fit docstring for all valid values

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data,
    time_limit=time_limit,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240925_022856"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.42 GB / 12.67 GB (90.1%)
Disk Space Avail:   180.45 GB / 225.83 GB (79.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
Fitted model: LightGBM/T1 ...
	0.37	 = Validation score   (accuracy)
	4.85s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	0.355	 = Validation score   (accuracy)
	2.98s	 = Training   runtime
	0.04s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	0.375	 = Validation score   (accuracy)
	1.1s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	0.36	 = Validation score   (accuracy)
	1.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	0.375	 = Validation score   (accuracy)
	1.15s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch 

+---------------------------------------------------+
| Configuration for experiment     NeuralNetTorch   |
+---------------------------------------------------+
| Search algorithm                 SearchGenerator  |
| Scheduler                        FIFOScheduler    |
| Number of trials                 5                |
+---------------------------------------------------+

View detailed results here: /content/AutogluonModels/ag-20240925_022856/models/NeuralNetTorch


Fitted model: NeuralNetTorch/e28ec7e3 ...
	0.355	 = Validation score   (accuracy)
	8.71s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: NeuralNetTorch/d6600f1a ...
	0.33	 = Validation score   (accuracy)
	11.3s	 = Training   runtime
	0.05s	 = Validation runtime
Fitted model: NeuralNetTorch/c2320597 ...
	0.31	 = Validation score   (accuracy)
	8.6s	 = Training   runtime
	0.03s	 = Validation runtime


Fitting model: WeightedEnsemble_L2 ... Training model for up to 119.65s of the 45.09s of remaining time.
	Ensemble Weights: {'LightGBM/T3': 0.938, 'NeuralNetTorch/c2320597': 0.062}
	0.38	 = Validation score   (accuracy)
	0.19s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 75.16s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 4497.8 rows/s (200 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240925_022856")


We again demonstrate how to use the trained models to predict on the test data.

In [4]:
y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

Predictions:   [' Other-service', ' Craft-repair', ' Exec-managerial', ' Sales', ' Other-service']


Use the following to view a summary of what happened during `fit()`. Now this command will show details of the hyperparameter-tuning process for each type of model:

In [5]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                     model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L2      0.380    accuracy       0.044466   9.886318                0.001168           0.188780            2       True          9
1              LightGBM/T3      0.375    accuracy       0.011362   1.096830                0.011362           1.096830            1       True          3
2              LightGBM/T5      0.375    accuracy       0.019516   1.148154                0.019516           1.148154            1       True          5
3              LightGBM/T1      0.370    accuracy       0.015371   4.845392                0.015371           4.845392            1       True          1
4              LightGBM/T4      0.360    accuracy       0.032285   1.359467                0.032285           1.359467            1       True          4
5  NeuralNetTo

In the above example, the predictive performance may be poor because we specified very little training to ensure quick runtimes.  You can call `fit()` multiple times while modifying the above settings to better understand how these choices affect performance outcomes. For example: you can comment out the `train_data.head` command or increase `subsample_size` to train using a larger dataset, increase the `num_epochs` and `num_boost_round` hyperparameters, and increase the `time_limit` (which you should do for all code in these tutorials).  To see more detailed output during the execution of `fit()`, you can also pass in the argument: `verbosity = 3`.

## Model ensembling with stacking/bagging

Beyond hyperparameter-tuning with a correctly-specified evaluation metric, two other methods to boost predictive performance are [bagging and stack-ensembling](https://arxiv.org/abs/2003.06505).  You'll often see performance improve if you specify `num_bag_folds` = 5-10, `num_stack_levels` = 1-3 in the call to `fit()`, but this will increase training times and memory/disk usage.

In [6]:
label = 'class'  # Now lets predict the "class" column (binary classification)
test_data_nolabel = test_data.drop(columns=[label])
y_test = test_data[label]
save_path = 'agModels-predictClass'  # folder where to store trained models

predictor = TabularPredictor(label=label, eval_metric=metric).fit(train_data,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1,
    hyperparameters = {'NN_TORCH': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}},  # last  argument is just for quick demo here, omit it in real applications
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240925_023013"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.57 GB / 12.67 GB (83.4%)
Disk Space Avail:   180.43 GB / 225.83 GB (79.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

You should not provide `tuning_data` when stacking/bagging, and instead provide all your available data as `train_data` (which AutoGluon will split in more intellgent ways). `num_bag_sets` controls how many times the k-fold bagging process is repeated to further reduce variance (increasing this may further boost accuracy but will substantially increase training times, inference latency, and memory/disk usage). Rather than manually searching for good bagging/stacking values yourself, AutoGluon will automatically select good values for you if you specify `auto_stack` instead:

In [7]:
# Lets also specify the "f1" metric
predictor = TabularPredictor(label=label, eval_metric='f1', path=save_path).fit(
    train_data, auto_stack=True,
    time_limit=30, hyperparameters={'FASTAI': {'num_epochs': 10}, 'GBM': {'num_boost_round': 200}}  # last 2 arguments are for quick demo, omit them in real applications
)
predictor.leaderboard(test_data)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.33 GB / 12.67 GB (81.5%)
Disk Space Avail:   180.43 GB / 225.83 GB (79.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Stack configuration (auto_stac

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.629437,0.68559,f1,1.025158,0.121284,36.629645,1.025158,0.121284,36.629645,1,True,1
1,WeightedEnsemble_L2,0.629437,0.68559,f1,1.029539,0.132617,36.645368,0.004380,0.011333,0.015723,2,True,2


Often stacking/bagging will produce superior accuracy than hyperparameter-tuning, but you may try combining both techniques (note: specifying `presets='best_quality'` in `fit()` simply sets `auto_stack=True`).

## Decision Threshold Calibration

Major metric score improvements can be achieved in binary classification for metrics such as `"f1"` and `"balanced_accuracy"` by adjusting the prediction decision threshold via `calibrate_decision_threshold` to a value other than the default 0.5.

Below is an example of the `"f1"` score achieved on the test data with and without calibrating the decision threshold:

In [8]:
print(f'Prior to calibration (predictor.decision_threshold={predictor.decision_threshold}):')
scores = predictor.evaluate(test_data)

calibrated_decision_threshold = predictor.calibrate_decision_threshold()
predictor.set_decision_threshold(calibrated_decision_threshold)

print(f'After calibration (predictor.decision_threshold={predictor.decision_threshold}):')
scores_calibrated = predictor.evaluate(test_data)

Prior to calibration (predictor.decision_threshold=0.5):


Calibrating decision threshold to optimize metric f1 | Checking 51 thresholds...
Calibrating decision threshold via fine-grained search | Checking 38 thresholds...
	Base Threshold: 0.500	| val: 0.6856
	Best Threshold: 0.500	| val: 0.6856


After calibration (predictor.decision_threshold=0.5):


In [9]:
for metric_name in scores:
    metric_score = scores[metric_name]
    metric_score_calibrated = scores_calibrated[metric_name]
    decision_threshold = predictor.decision_threshold
    print(f'decision_threshold={decision_threshold:.3f}\t| metric="{metric_name}"'
          f'\n\ttest_score uncalibrated: {metric_score:.4f}'
          f'\n\ttest_score   calibrated: {metric_score_calibrated:.4f}'
          f'\n\ttest_score        delta: {metric_score_calibrated-metric_score:.4f}')

decision_threshold=0.500	| metric="f1"
	test_score uncalibrated: 0.6294
	test_score   calibrated: 0.6294
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="accuracy"
	test_score uncalibrated: 0.8472
	test_score   calibrated: 0.8472
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="balanced_accuracy"
	test_score uncalibrated: 0.7438
	test_score   calibrated: 0.7438
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="mcc"
	test_score uncalibrated: 0.5457
	test_score   calibrated: 0.5457
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="roc_auc"
	test_score uncalibrated: 0.8990
	test_score   calibrated: 0.8990
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="precision"
	test_score uncalibrated: 0.7411
	test_score   calibrated: 0.7411
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="recall"
	test_score uncalibrated: 0.5470
	test_score   calibrated: 0.5470
	test_score        delta: 0.0

Notice that calibrating for "f1" majorly improved the "f1" metric score, as well as the "balanced_accuracy" score, it harmed the "accuracy" score. Threshold calibration will often result in a tradeoff between performance on different metrics, and the user should keep this in mind.

Instead of calibrating for "f1" specifically, we can calibrate for any metric if we want to maximize the score of that metric:

In [10]:
predictor.set_decision_threshold(0.5)  # Reset decision threshold
for metric_name in ['f1', 'balanced_accuracy', 'mcc']:
    metric_score = predictor.evaluate(test_data, silent=True)[metric_name]
    calibrated_decision_threshold = predictor.calibrate_decision_threshold(metric=metric_name, verbose=False)
    metric_score_calibrated = predictor.evaluate(
        test_data, decision_threshold=calibrated_decision_threshold, silent=True
    )[metric_name]
    print(f'decision_threshold={calibrated_decision_threshold:.3f}\t| metric="{metric_name}"'
          f'\n\ttest_score uncalibrated: {metric_score:.4f}'
          f'\n\ttest_score   calibrated: {metric_score_calibrated:.4f}'
          f'\n\ttest_score        delta: {metric_score_calibrated-metric_score:.4f}')

decision_threshold=0.500	| metric="f1"
	test_score uncalibrated: 0.6294
	test_score   calibrated: 0.6294
	test_score        delta: 0.0000
decision_threshold=0.250	| metric="balanced_accuracy"
	test_score uncalibrated: 0.7438
	test_score   calibrated: 0.8120
	test_score        delta: 0.0682
decision_threshold=0.500	| metric="mcc"
	test_score uncalibrated: 0.5457
	test_score   calibrated: 0.5457
	test_score        delta: 0.0000


Instead of calibrating the decision threshold post-fit, you can have it automatically occur during the fit call by specifying the fit parameter `predictor.fit(..., calibrate_decision_threshold=True)`.

Additional usage examples are below:

In [11]:
# Will use the decision_threshold specified in `predictor.decision_threshold`, can be set via `predictor.set_decision_threshold`
# y_pred = predictor.predict(test_data)
# y_pred_08 = predictor.predict(test_data, decision_threshold=0.8)  # Specify a specific threshold to use only for this predict

# y_pred_proba = predictor.predict_proba(test_data)
# y_pred = predictor.predict_from_proba(y_pred_proba)  # Identical output to calling .predict(test_data)
# y_pred_08 = predictor.predict_from_proba(y_pred_proba, decision_threshold=0.8)  # Identical output to calling .predict(test_data, decision_threshold=0.8)

## Prediction options (inference)

Even if you've started a new Python session since last calling `fit()`, you can still load a previously trained predictor from disk:

In [12]:
predictor = TabularPredictor.load(save_path)  # `predictor.path` is another way to get the relative path needed to later load predictor.

Above `save_path` is the same folder previously passed to `TabularPredictor`, in which all the trained models have been saved. You can train easily models on one machine and deploy them on another. Simply copy the `save_path` folder to the new machine and specify its new path in `TabularPredictor.load()`.

To find out the required feature columns to make predictions, call `predictor.features()`:

In [13]:
predictor.features()

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country']

We can make a prediction on an individual example rather than a full dataset:

In [14]:
datapoint = test_data_nolabel.iloc[[0]]  # Note: .iloc[0] won't work because it returns pandas Series instead of DataFrame
print(datapoint)
predictor.predict(datapoint)

   age workclass  fnlwgt education  education-num       marital-status  \
0   31   Private  169085      11th              7   Married-civ-spouse   

  occupation relationship    race      sex  capital-gain  capital-loss  \
0      Sales         Wife   White   Female             0             0   

   hours-per-week  native-country  
0              20   United-States  


,class
0,<=50K


To output predicted class probabilities instead of predicted classes, you can use:

In [15]:
predictor.predict_proba(datapoint)  # returns a DataFrame that shows which probability corresponds to which class

,<=50K,>50K
0,0.951059,0.048941


By default, `predict()` and `predict_proba()` will utilize the model that AutoGluon thinks is most accurate, which is usually an ensemble of many individual models. Here's how to see which model this is:

In [16]:
predictor.model_best

'WeightedEnsemble_L2'

We can instead specify a particular model to use for predictions (e.g. to reduce inference latency). Note that a 'model' in AutoGluon may refer to, for example, a single Neural Network, a bagged ensemble of many Neural Network copies trained on different training/validation splits, a weighted ensemble that aggregates the predictions of many other models, or a stacker model that operates on predictions output by other models. This is akin to viewing a Random Forest as one 'model' when it is in fact an ensemble of many decision trees.


Before deciding which model to use, let's evaluate all of the models AutoGluon has previously trained on our test data:

In [17]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.629437,0.68559,f1,0.795546,0.121284,36.629645,0.795546,0.121284,36.629645,1,True,1
1,WeightedEnsemble_L2,0.629437,0.68559,f1,0.797815,0.132617,36.645368,0.002269,0.011333,0.015723,2,True,2


The leaderboard shows each model's predictive performance on the test data (`score_test`) and validation data (`score_val`), as well as the time required to: produce predictions for the test data (`pred_time_val`), produce predictions on the validation data (`pred_time_val`), and train only this model (`fit_time`). Below, we show that a leaderboard can be produced without new data (just uses the data previously reserved for validation inside `fit`) and can display extra information about each model:

In [18]:
predictor.leaderboard(extra_info=True)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order,...,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,LightGBM_BAG_L1,0.68559,f1,0.121284,36.629645,0.121284,36.629645,1,True,1,...,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[capital-loss, marital-status, fnlwgt, native-...",None,"{'learning_rate': 0.05, 'num_boost_round': 200}",{'num_boost_round': 83},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[],[WeightedEnsemble_L2]
1,WeightedEnsemble_L2,0.68559,f1,0.132617,36.645368,0.011333,0.015723,2,True,2,...,"{'use_orig_features': False, 'max_base_models'...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[LightGBM_BAG_L1],None,"{'ensemble_size': 25, 'subsample_size': 1000000}",{'ensemble_size': 1},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[LightGBM_BAG_L1],[]


The expanded leaderboard shows properties like how many features are used by each model (`num_features`), which other models are ancestors whose predictions are required inputs for each model (`ancestors`), and how much memory each model and all its ancestors would occupy if simultaneously persisted (`memory_size_w_ancestors`). See the [leaderboard documentation](../../api/autogluon.tabular.TabularPredictor.leaderboard.rst) for full details.

To show scores for other metrics, you can specify the `extra_metrics` argument when passing in `test_data`:

In [19]:
predictor.leaderboard(test_data, extra_metrics=['accuracy', 'balanced_accuracy', 'log_loss'])

,model,score_test,accuracy,balanced_accuracy,log_loss,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.629437,0.84717,0.743784,-0.334022,0.68559,f1,0.766822,0.121284,36.629645,0.766822,0.121284,36.629645,1,True,1
1,WeightedEnsemble_L2,0.629437,0.84717,0.743784,-0.334022,0.68559,f1,0.769127,0.132617,36.645368,0.002305,0.011333,0.015723,2,True,2


Notice that `log_loss` scores are negative.
This is because metrics in AutoGluon are always shown in `higher_is_better` form.
This means that metrics such as `log_loss` and `root_mean_squared_error` will have their signs FLIPPED, and values will be negative.
This is necessary to avoid the user needing to know the metric to understand if higher is better when looking at leaderboard.

One additional caveat: It is possible that `log_loss` values can be `-inf` when computed via `extra_metrics`.
This is because the models were not optimized with `log_loss` in mind during training and
may have prediction probabilities giving a class `0` (particularly common with K-Nearest-Neighbors models).
Because `log_loss` gives infinite error when the correct class was given `0` probability, this results in a score of `-inf`.
It is therefore recommended that `log_loss` should not be used as a secondary metric to determine model quality.
Either use `log_loss` as the `eval_metric` or avoid it altogether.

Here's how to specify a particular model to use for prediction instead of AutoGluon's default model-choice:

In [20]:
i = 0  # index of model to use
model_to_use = predictor.model_names()[i]
model_pred = predictor.predict(datapoint, model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred.iloc[0]))

Prediction from LightGBM_BAG_L1 model:  <=50K


We can easily access various information about the trained predictor or a particular model:

In [21]:
all_models = predictor.model_names()
model_to_use = all_models[i]
specific_model = predictor._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = predictor.info()

The `predictor` also remembers what metric predictions should be evaluated with, which can be done with ground truth labels as follows:

In [22]:
y_pred_proba = predictor.predict_proba(test_data_nolabel)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred_proba)

Since the label columns remains in the `test_data` DataFrame, we can instead use the shorthand:

In [23]:
perf = predictor.evaluate(test_data)

## Interpretability (feature importance)

To better understand our trained predictor, we can estimate the overall importance of each feature:

In [24]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 14 features using 5000 rows with 5 shuffle sets...
	35.08s	= Expected runtime (7.02s per shuffle set)
	31.45s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
marital-status,0.117047,0.009311,4.765349e-06,5,0.136219,0.097875
capital-gain,0.083102,0.004023,6.569424e-07,5,0.091385,0.074819
education-num,0.071497,0.006756,9.453208e-06,5,0.085407,0.057587
age,0.055806,0.009717,1.059856e-04,5,0.075814,0.035799
occupation,0.055400,0.013509,3.925863e-04,5,0.083214,0.027585
relationship,0.020842,0.006809,1.192225e-03,5,0.034861,0.006822
hours-per-week,0.018326,0.006083,1.264850e-03,5,0.030850,0.005802
capital-loss,0.003712,0.002108,8.495976e-03,5,0.008052,-0.000628
education,0.000719,0.001339,1.480534e-01,5,0.003477,-0.002038
native-country,0.000000,0.000000,5.000000e-01,5,0.000000,0.000000


Computed via [permutation-shuffling](https://explained.ai/rf-importance/), these feature importance scores quantify the drop in predictive performance (of the already trained predictor) when one column's values are randomly shuffled across rows. The top features in this list contribute most to AutoGluon's accuracy (for predicting when/if a patient will be readmitted to the hospital). Features with non-positive importance score hardly contribute to the predictor's accuracy, or may even be actively harmful to include in the data (consider removing these features from your data and calling `fit` again). These scores facilitate interpretability of the predictor's global behavior (which features it relies on for *all* predictions).
To get [local explanations](https://christophm.github.io/interpretable-ml-book/taxonomy-of-interpretability-methods.html) regarding which features influence a *particular* prediction, check out the [example notebooks](https://github.com/autogluon/autogluon/tree/master/examples/tabular/interpret) for explaining particular AutoGluon predictions using [Shapely values](https://github.com/slundberg/shap/).

Before making judgement on if AutoGluon is more or less interpretable than another solution, we recommend reading [The Mythos of Model Interpretability](https://dl.acm.org/doi/pdf/10.1145/3236386.3241340) by Zachary Lipton, which covers why often-claimed interpretable models such as trees and linear models are rarely meaningfully more interpretable than more advanced models.

## Accelerating inference

We describe multiple ways to reduce the time it takes for AutoGluon to produce predictions.

Before providing code examples, it is important to understand that
there are several ways to accelerate inference in AutoGluon. The table below lists the options in order of priority.

| Optimization                      | Inference Speedup                                                                                     | Cost                              | Notes                                                                                                                                                                                  |
|:----------------------------------|:------------------------------------------------------------------------------------------------------|:----------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| refit_full                        | At least 8x+, up to 160x (requires bagging)                                                           | -Quality, +FitTime                | Only provides speedup with bagging enabled.                                                                                                                                            |
| persist                           | Up to 10x in online-inference                                                                         | ++MemoryUsage                     | If memory is not sufficient to persist model, speedup is not gained. Speedup is most effective in online-inference and is not relevant in batch inference.                             |
| infer_limit                       | Configurable, ~up to 50x                                                                              | -Quality (Relative to speedup)    | If bagging is enabled, always use refit_full if using infer_limit.                                                                                                                     |
| distill                           | ~Equals combined speedup of refit_full and infer_limit set to extreme values                          | --Quality, ++FitTime              | Not compatible with refit_full and infer_limit.                                                                                                                                        |
| feature pruning                   | Typically at most 1.5x. More if willing to lower quality significantly.                               | -Quality?, ++FitTime              | Dependent on the existence of unimportant features in data. Call `predictor.feature_importance(test_data)` to gauge which features could be removed.                                   |
| use faster hardware               | Usually at most 3x. Depends on hardware (ignoring GPU).                                               | +Hardware                         | As an example, an EC2 c6i.2xlarge is ~1.6x faster than an m5.2xlarge for a similar price. Laptops in particular might be slow compared to cloud instances.                             |
| manual hyperparameters adjustment | Usually at most 2x assuming infer_limit is already specified.                                         | ---Quality?, +++UserMLExpertise   | Can be very complicated and is not recommended. Potential ways to get speedups this way is to reduce the number of trees in LightGBM, XGBoost, CatBoost, RandomForest, and ExtraTrees. |
| manual data preprocessing         | Usually at most 1.2x assuming all other optimizations are specified and setting is online-inference.  | ++++UserMLExpertise, ++++UserCode | Only relevant for online-inference. This is not recommended as AutoGluon's default preprocessing is highly optimized.                                                                  |

If bagging is enabled (num_bag_folds>0 or num_stack_levels>0 or using 'best_quality' preset), the order of inference optimizations should be:  
1. refit_full  
2. persist  
3. infer_limit  

If bagging is not enabled (num_bag_folds=0, num_stack_levels=0), the order of inference optimizations should be:  
1. persist  
2. infer_limit  

If following these recommendations does not lead to a sufficiently fast model, you may consider the more advanced options in the table.

### Keeping models in memory

By default, AutoGluon loads models into memory one at a time and only when they are needed for prediction. This strategy is robust for large stacked/bagged ensembles, but leads to slower prediction times. If you plan to repeatedly make predictions (e.g. on new datapoints one at a time rather than one large test dataset), you can first specify that all models required for inference should be loaded into memory as follows:

In [25]:
predictor.persist()

num_test = 20
preds = np.array(['']*num_test, dtype='object')
for i in range(num_test):
    datapoint = test_data_nolabel.iloc[[i]]
    pred_numpy = predictor.predict(datapoint, as_pandas=False)
    preds[i] = pred_numpy[0]

perf = predictor.evaluate_predictions(y_test[:num_test], preds, auxiliary_metrics=True)
print("Predictions: ", preds)

predictor.unpersist()  # free memory by clearing models, future predict() calls will load models from disk

Persisting 2 models in memory. Models will require 0.02% of memory.
Unpersisted 2 models: ['LightGBM_BAG_L1', 'WeightedEnsemble_L2']


Predictions:  [' <=50K' ' <=50K' ' >50K' ' <=50K' ' <=50K' ' >50K' ' >50K' ' >50K'
 ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K'
 ' <=50K' ' >50K' ' >50K' ' <=50K']


['LightGBM_BAG_L1', 'WeightedEnsemble_L2']

You can alternatively specify a particular model to persist via the `models` argument of `persist()`, or simply set `models='all'` to simultaneously load every single model that was trained during `fit`.

### Inference speed as a fit constraint

If you know your latency constraint prior to fitting the predictor, you can specify it explicitly as a fit argument.
AutoGluon will then automatically train models in a fashion that attempts to satisfy the constraint.

This constraint has two components: `infer_limit` and `infer_limit_batch_size`:  
- `infer_limit` is the time in seconds to predict 1 row of data.
For example, `infer_limit=0.05` means 50 ms per row of data,
or 20 rows / second throughput.  
- `infer_limit_batch_size` is the amount of rows passed at once to predict when calculating per-row speed.
This is very important because `infer_limit_batch_size=1` (online-inference) is highly suboptimal as
various operations have a fixed cost overhead regardless of data size. If you can pass your test data in bulk,
you should specify `infer_limit_batch_size=10000`.

In [26]:
# At most 0.05 ms per row (20000 rows per second throughput)
infer_limit = 0.00005
# adhere to infer_limit with batches of size 10000 (batch-inference, easier to satisfy infer_limit)
infer_limit_batch_size = 10000
# adhere to infer_limit with batches of size 1 (online-inference, much harder to satisfy infer_limit)
# infer_limit_batch_size = 1  # Note that infer_limit<0.02 when infer_limit_batch_size=1 can be difficult to satisfy.
predictor_infer_limit = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data=train_data,
    time_limit=30,
    infer_limit=infer_limit,
    infer_limit_batch_size=infer_limit_batch_size,
)

# NOTE: If bagging was enabled, it is important to call refit_full at this stage.
#  infer_limit assumes that the user will call refit_full after fit.
# predictor_infer_limit.refit_full()

# NOTE: To align with inference speed calculated during fit, models must be persisted.
predictor_infer_limit.persist()
# Below is an optimized version that only persists the minimum required models for prediction.
# predictor_infer_limit.persist('best')

predictor_infer_limit.leaderboard()

No path specified. Models will be saved in: "AutogluonModels/ag-20240925_023342"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.67 GB / 12.67 GB (84.2%)
Disk Space Avail:   180.43 GB / 225.83 GB (79.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.860,accuracy,0.006047,6.647030,0.006047,6.647030,1,True,7
1,WeightedEnsemble_L2,0.860,accuracy,0.008084,6.770669,0.002037,0.123639,2,True,14
2,LightGBMXT,0.850,accuracy,0.006023,0.501406,0.006023,0.501406,1,True,3
3,NeuralNetTorch,0.850,accuracy,0.027420,13.171381,0.027420,13.171381,1,True,12
4,XGBoost,0.845,accuracy,0.008106,0.494583,0.008106,0.494583,1,True,11
5,LightGBM,0.840,accuracy,0.005626,0.506490,0.005626,0.506490,1,True,4
6,NeuralNetFastAI,0.840,accuracy,0.015012,2.302890,0.015012,2.302890,1,True,10
7,RandomForestGini,0.840,accuracy,0.090276,1.206081,0.090276,1.206081,1,True,5
8,RandomForestEntr,0.835,accuracy,0.083714,1.184451,0.083714,1.184451,1,True,6
9,ExtraTreesEntr,0.820,accuracy,0.091184,1.143876,0.091184,1.143876,1,True,9


Now we can test the inference speed of the final model and check if it satisfies the inference constraints.

In [27]:
test_data_batch = test_data.sample(infer_limit_batch_size, replace=True, ignore_index=True)

import time
time_start = time.time()
predictor_infer_limit.predict(test_data_batch)
time_end = time.time()

infer_time_per_row = (time_end - time_start) / len(test_data_batch)
rows_per_second = 1 / infer_time_per_row
infer_time_per_row_ratio = infer_time_per_row / infer_limit
is_constraint_satisfied = infer_time_per_row_ratio <= 1

print(f'Model is able to predict {round(rows_per_second, 1)} rows per second. (User-specified Throughput = {1 / infer_limit})')
print(f'Model uses {round(infer_time_per_row_ratio * 100, 1)}% of infer_limit time per row.')
print(f'Model satisfies inference constraint: {is_constraint_satisfied}')

Model is able to predict 62515.8 rows per second. (User-specified Throughput = 20000.0)
Model uses 32.0% of infer_limit time per row.
Model satisfies inference constraint: True


### Using smaller ensemble or faster model for prediction

Without having to retrain any models, one can construct alternative ensembles that aggregate individual models' predictions with different weighting schemes. These ensembles become smaller (and hence faster for prediction) if they assign nonzero weight to less models. You can produce a wide variety of ensembles with different accuracy-speed tradeoffs like this:

In [28]:
additional_ensembles = predictor.fit_weighted_ensemble(expand_pareto_frontier=True)
print("Alternative ensembles you can use for prediction:", additional_ensembles)

predictor.leaderboard(only_pareto_frontier=True)

Fitting model: WeightedEnsemble_L2Best ...
	Ensemble Weights: {'LightGBM_BAG_L1': 1.0}
	0.6856	 = Validation score   (f1)
	0.04s	 = Training   runtime
	0.02s	 = Validation runtime


Alternative ensembles you can use for prediction: ['WeightedEnsemble_L2Best']


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.68559,f1,0.121284,36.629645,0.121284,36.629645,1,True,1


The resulting leaderboard will contain the most accurate model for a given inference-latency. You can select whichever model exhibits acceptable latency from the leaderboard and use it for prediction.

In [29]:
model_for_prediction = additional_ensembles[0]
predictions = predictor.predict(test_data, model=model_for_prediction)
predictor.delete_models(models_to_delete=additional_ensembles, dry_run=False)  # delete these extra models so they don't affect rest of tutorial

Deleting model WeightedEnsemble_L2Best. All files under agModels-predictClass/models/WeightedEnsemble_L2Best will be removed.


### Collapsing bagged ensembles via refit_full

For an ensemble predictor trained with bagging (as done above), recall there are ~10 bagged copies of each individual model trained on different train/validation folds. We can collapse this bag of ~10 models into a single model that's fit to the full dataset, which can greatly reduce its memory/latency requirements (but may also reduce accuracy). Below we refit such a model for each original model but you can alternatively do this for just a particular model by specifying the `model` argument of `refit_full()`.

In [30]:
refit_model_map = predictor.refit_full()
print("Name of each refit-full model corresponding to a previous bagged ensemble:")
print(refit_model_map)
predictor.leaderboard(test_data)

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: LightGBM_BAG_L1_FULL ...
	0.47s	 = Training   runtime
Fitting model: WeightedEnsemble_L2_FULL | Skipping fit via cloning parent ...
	Ensemble Weights: {'LightGBM_BAG_L1': 1.0}
	0.02s	 = Training   runtime
Updated best model to "WeightedEnsemble_L2_FULL" (Previously "WeightedEnsemble_L2"). AutoGluon will default to using "WeightedEnsemble_L2_FULL" for predict() and predict_proba().
Refit complete, total runtime = 0.55s ... Best model: "WeightedEnsemble_L2_FULL"


Name of each refit-full model corresponding to a previous bagged ensemble:
{'LightGBM_BAG_L1': 'LightGBM_BAG_L1_FULL', 'WeightedEnsemble_L2': 'WeightedEnsemble_L2_FULL'}


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1_FULL,0.634860,NaN,f1,0.127968,NaN,0.467451,0.127968,NaN,0.467451,1,True,3
1,WeightedEnsemble_L2_FULL,0.634860,NaN,f1,0.131647,NaN,0.483174,0.003679,NaN,0.015723,2,True,4
2,LightGBM_BAG_L1,0.629437,0.68559,f1,0.906079,0.121284,36.629645,0.906079,0.121284,36.629645,1,True,1
3,WeightedEnsemble_L2,0.629437,0.68559,f1,0.909834,0.132617,36.645368,0.003756,0.011333,0.015723,2,True,2


This adds the refit-full models to the leaderboard and we can opt to use any of them for prediction just like any other model. Note `pred_time_test` and `pred_time_val` list the time taken to produce predictions with each model (in seconds) on the test/validation data. Since the refit-full models were trained using all of the data, there is no internal validation score (`score_val`) available for them. You can also call `refit_full()` with non-bagged models to refit the same models to your full dataset (there won't be memory/latency gains in this case but test accuracy may improve).

### Model distillation

While computationally-favorable, single individual models will usually have lower accuracy than weighted/stacked/bagged ensembles. [Model Distillation](https://arxiv.org/abs/2006.14284) offers one way to retain the computational benefits of a single model, while enjoying some of the accuracy-boost that comes with ensembling. The idea is to train the individual model (which we can call the student) to mimic the predictions of the full stack ensemble (the teacher). Like `refit_full()`, the `distill()` function will produce additional models we can opt to use for prediction.

In [31]:
student_models = predictor.distill(time_limit=30)  # specify much longer time limit in real applications
print(student_models)
preds_student = predictor.predict(test_data_nolabel, model=student_models[0])
print(f"predictions from {student_models[0]}:", list(preds_student)[:5])
predictor.leaderboard(test_data)

Distilling with teacher='WeightedEnsemble_L2_FULL', teacher_preds=soft, augment_method=spunge ...
SPUNGE: Augmenting training data with 4000 synthetic samples for distillation...
Distilling with each of these student models: ['LightGBM_DSTL', 'CatBoost_DSTL', 'RandomForestMSE_DSTL', 'NeuralNetTorch_DSTL']
Fitting 4 L1 models ...
Fitting model: LightGBM_DSTL ... Training model for up to 30.0s of the 29.99s of remaining time.
		pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: workclass: object, education: object, marital-status: object, occupation: object, relationship: object, race: object, native-country: object
Detailed Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/autogluon/core/trainer/abstract_trainer.py", line 1904, in _train_and_save
    model = self._train_single(X, y, model, X_val, y_val, total_resources=total_resources, **model_fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/autogluon/core/trai

['RandomForestMSE_DSTL', 'WeightedEnsemble_L2_DSTL']
predictions from RandomForestMSE_DSTL: [' <=50K', ' <=50K', ' >50K', ' <=50K', ' <=50K']


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1_FULL,0.634860,NaN,f1,0.095961,NaN,0.467451,0.095961,NaN,0.467451,1,True,3
1,WeightedEnsemble_L2_FULL,0.634860,NaN,f1,0.098186,NaN,0.483174,0.002225,NaN,0.015723,2,True,4
2,LightGBM_BAG_L1,0.629437,0.685590,f1,0.797410,0.121284,36.629645,0.797410,0.121284,36.629645,1,True,1
3,WeightedEnsemble_L2,0.629437,0.685590,f1,0.799692,0.132617,36.645368,0.002283,0.011333,0.015723,2,True,2
4,RandomForestMSE_DSTL,0.607631,0.629213,mean_squared_error,0.544061,0.140608,7.664801,0.544061,0.140608,7.664801,1,True,5
5,WeightedEnsemble_L2_DSTL,0.607631,0.629213,mean_squared_error,0.546880,0.141998,7.687495,0.002819,0.001390,0.022694,2,True,6


### Faster presets or hyperparameters

Instead of trying to speed up a cumbersome trained model at prediction time, if you know inference latency or memory will be an issue at the outset, then you can adjust the training process accordingly to ensure `fit()` does not produce unwieldy models.

One option is to specify more lightweight `presets`:

In [32]:
presets = ['good_quality', 'optimize_for_deployment']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, presets=presets, time_limit=300)

No path specified. Models will be saved in: "AutogluonModels/ag-20240925_023441"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.28 GB / 12.67 GB (81.1%)
Disk Space Avail:   180.30 GB / 225.83 GB (79.8%)
Presets specified: ['good_quality', 'optimize_for_deployment']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag

Another option is to specify more lightweight hyperparameters:

In [33]:
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, hyperparameters='very_light', time_limit=30)

No path specified. Models will be saved in: "AutogluonModels/ag-20240925_023950"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.58 GB / 12.67 GB (75.6%)
Disk Space Avail:   180.29 GB / 225.83 GB (79.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : F

Here you can set `hyperparameters` to either 'light', 'very_light', or 'toy' to obtain progressively smaller (but less accurate) models and predictors. Advanced users may instead try manually specifying particular models' hyperparameters in order to make them faster/smaller.

Finally, you may also exclude specific unwieldy models from being trained at all. Below we exclude models that tend to be slower (K Nearest Neighbors, Neural Networks):

In [34]:
excluded_model_types = ['KNN', 'NN_TORCH']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, excluded_model_types=excluded_model_types, time_limit=30)

No path specified. Models will be saved in: "AutogluonModels/ag-20240925_024007"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.24 GB / 12.67 GB (80.8%)
Disk Space Avail:   180.29 GB / 225.83 GB (79.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

### (Advanced) Cache preprocessed data

If you are repeatedly predicting on the same data you can cache the preprocessed version of the data and
directly send the preprocessed data to `predictor.predict` for faster inference:

Note that this is only useful in situations where you are repeatedly predicting on the same data.
If this significantly speeds up your use-case, consider whether your current approach makes sense
or if a cache on the predictions is a better solution.

### (Advanced) Disable preprocessing

If you would rather do data preprocessing outside of TabularPredictor,
you can disable TabularPredictor's preprocessing entirely via:

Be warned that this removes ALL guardrails on data sanitization.
It is very likely that you will run into errors doing this unless you are very familiar with AutoGluon.

One instance where this can be helpful is if you have many problems
that re-use the exact same data with the exact same features. If you had 30 tasks that re-use the same features,
you could fit a `autogluon.features` feature generator once on the data, and then when you need to
predict on the 30 tasks, preprocess the data only once and then send the preprocessed data to all 30 predictors.


## If you encounter memory issues

To reduce memory usage during training, you may try each of the following strategies individually or combinations of them (these may harm accuracy):

- In `fit()`, set `excluded_model_types = ['KNN', 'XT' ,'RF']` (or some subset of these models).
- Try different `presets` in `fit()`.
- In `fit()`, set `hyperparameters = 'light'` or `hyperparameters = 'very_light'`.
- Text fields in your table require substantial memory for N-gram featurization. To mitigate this in `fit()`, you can either: (1) add `'ignore_text'` to your `presets` list (to ignore text features), or (2) specify the argument:

```
from sklearn.feature_extraction.text import CountVectorizer
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
feature_generator = AutoMLPipelineFeatureGenerator(vectorizer=CountVectorizer(min_df=30, ngram_range=(1, 3), max_features=MAX_NGRAM, dtype=np.uint8))
```


for example using `MAX_NGRAM = 1000` (try various values under 10000 to reduce the number of N-gram features used to represent each text field)

In addition to reducing memory usage, many of the above strategies can also be used to reduce training times.

To reduce memory usage during inference:

- If trying to produce predictions for a large test dataset, break the test data into smaller chunks as demonstrated in [FAQ](tabular-faq.ipynb).

- If models have been previously persisted in memory but inference-speed is not a major concern, call `predictor.unpersist()`.

- If models have been previously persisted in memory, bagging was used in `fit()`, and inference-speed is a concern: call `predictor.refit_full()` and use one of the refit-full models for prediction (ensure this is the only model persisted in memory).



## If you encounter disk space issues

To reduce disk usage, you may try each of the following strategies individually or combinations of them:

- Make sure to delete all `predictor.path` folders from previous `fit()` runs! These can eat up your free space if you call `fit()` many times. If you didn't specify `path`, AutoGluon still automatically saved its models to a folder called: "AutogluonModels/ag-[TIMESTAMP]", where TIMESTAMP records when `fit()` was called, so make sure to also delete these folders if you run low on free space.

- Call `predictor.save_space()` to delete auxiliary files produced during `fit()`.

- Call `predictor.delete_models(models_to_keep='best', dry_run=False)` if you only intend to use this predictor for inference going forward (will delete files required for non-prediction-related functionality like `fit_summary`).

- In `fit()`, you can add `'optimize_for_deployment'` to the `presets` list, which will automatically invoke the previous two strategies after training.

- Most of the above strategies to reduce memory usage will also reduce disk usage (but may harm accuracy).


## References

The following paper describes how AutoGluon internally operates on tabular data:

Erickson et al. [AutoGluon-Tabular: Robust and Accurate AutoML for Structured Data](https://arxiv.org/abs/2003.06505). *Arxiv*, 2020.

## Next Steps

If you are interested in deployment optimization, refer to the [Predicting Columns in a Table - Deployment Optimization](advanced/tabular-deployment.ipynb) tutorial.